**Data Exploration**
---




In [99]:
import pandas as pd
data = pd.read_excel('/content/drive/My Drive/Master thesis/DataSet-forUsing/3-hatespeech dataset _ nuha_ Are they Our Brothers paper  - Sheet1.xlsx', 
                   index_col = 0
                  )
data.columns = ['label', 'text']
data.head()

,label,text
id,,
930000000000000000,not-hate,مؤسسة أرشيف المغرب تتسلم وثائق عن ذاكرة اليهود...
930000000000000000,not-hate,مفتي السعودية حماس إرهابية وقتال اليهود حرام ش...
930000000000000000,hate,أمراء ال سعود اليهود يخوضون حربا عن الصهيونيه
930000000000000000,hate,ياحكام العرب أقسم باالله عذاب الله واقع عليكم ...
930000000000000000,hate,عندما كنت بأمريكا حتم علي عملي ان احتك بفتيات ...


In [100]:
#data.shape
data.describe

<bound method NDFrame.describe of                        label                                               text
id                                                                             
930000000000000000  not-hate  مؤسسة أرشيف المغرب تتسلم وثائق عن ذاكرة اليهود...
930000000000000000  not-hate  مفتي السعودية حماس إرهابية وقتال اليهود حرام ش...
930000000000000000      hate      أمراء ال سعود اليهود يخوضون حربا عن الصهيونيه
930000000000000000      hate  ياحكام العرب أقسم باالله عذاب الله واقع عليكم ...
930000000000000000      hate  عندما كنت بأمريكا حتم علي عملي ان احتك بفتيات ...
...                      ...                                                ...
958342083311435777      hate  سِجِسْتان:\n.\nقال ياقوت (ت٦٢٦هـ) عنها :\n"ناح...
958341695892017154  not-hate  حسبي الله لا إله إلا هو عليه توكلت وهو ربّ الع...
958341568469094400      hate  العرب البقرة الحلوب للمستعمر الغربي الصليبي وا...
958339664036945922  not-hate  اتَّبِعِ الْحَقَّ وَلَا تُبَالِ ، وَاحْبِبِ ال...
958339

# Data Cleaning
---








1.   Remove punctuation marks by using Regex.


In [101]:
import re,string

def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', str(text))
    return text_nopunct
data['Text_Without_punct'] = data['text'].apply(lambda x: remove_punct(x))

2.   tokenize the comments by using NLTK’s word_tokenize


In [104]:
import nltk
#nltk.download('punkt')
from nltk import word_tokenize
tokens = [word_tokenize(sen) for sen in data.Text_Without_punct]


3.  lower case the data



In [105]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

4. Remove stop words from data using NLTK’s stopwords.

In [106]:
# nltk.download('stopwords')
from nltk.corpus import stopwords
stoplist = stopwords.words('arabic')
def removeStopWords(tokens): 
    return [word for word in tokens if word not in stoplist]
filtered_words = [removeStopWords(sen) for sen in lower_tokens]
data['Text_Final'] = [' '.join(sen) for sen in filtered_words]
data['tokens'] = filtered_words

In [107]:
data

,label,text,Text_Without_punct,Text_Final,tokens
id,,,,,
930000000000000000,not-hate,مؤسسة أرشيف المغرب تتسلم وثائق عن ذاكرة اليهود...,مؤسسة أرشيف المغرب تتسلم وثائق عن ذاكرة اليهود...,مؤسسة أرشيف المغرب تتسلم وثائق ذاكرة اليهود ال...,"[مؤسسة, أرشيف, المغرب, تتسلم, وثائق, ذاكرة, ال..."
930000000000000000,not-hate,مفتي السعودية حماس إرهابية وقتال اليهود حرام ش...,مفتي السعودية حماس إرهابية وقتال اليهود حرام ش...,مفتي السعودية حماس إرهابية وقتال اليهود حرام ش...,"[مفتي, السعودية, حماس, إرهابية, وقتال, اليهود,..."
930000000000000000,hate,أمراء ال سعود اليهود يخوضون حربا عن الصهيونيه,أمراء ال سعود اليهود يخوضون حربا عن الصهيونيه,أمراء ال سعود اليهود يخوضون حربا الصهيونيه,"[أمراء, ال, سعود, اليهود, يخوضون, حربا, الصهيو..."
930000000000000000,hate,ياحكام العرب أقسم باالله عذاب الله واقع عليكم ...,ياحكام العرب أقسم باالله عذاب الله واقع عليكم ...,ياحكام العرب أقسم باالله عذاب الله واقع عليكم ...,"[ياحكام, العرب, أقسم, باالله, عذاب, الله, واقع..."
930000000000000000,hate,عندما كنت بأمريكا حتم علي عملي ان احتك بفتيات ...,عندما كنت بأمريكا حتم علي عملي ان احتك بفتيات ...,عندما كنت بأمريكا حتم علي عملي ان احتك بفتيات ...,"[عندما, كنت, بأمريكا, حتم, علي, عملي, ان, احتك..."
...,...,...,...,...,...
958342083311435777,hate,"سِجِسْتان:\n.\nقال ياقوت (ت٦٢٦هـ) عنها :\n""ناح...",سِجِسْتان\n\nقال ياقوت ت٦٢٦هـ عنها \nناحية كبي...,سِجِسْتان قال ياقوت ت٦٢٦هـ عنها ناحية كبيرة وو...,"[سِجِسْتان, قال, ياقوت, ت٦٢٦هـ, عنها, ناحية, ك..."
958341695892017154,not-hate,حسبي الله لا إله إلا هو عليه توكلت وهو ربّ الع...,حسبي الله لا إله إلا هو عليه توكلت وهو ربّ الع...,حسبي الله إله توكلت ربّ العرش العظيممن قالها ي...,"[حسبي, الله, إله, توكلت, ربّ, العرش, العظيممن,..."
958341568469094400,hate,العرب البقرة الحلوب للمستعمر الغربي الصليبي وا...,العرب البقرة الحلوب للمستعمر الغربي الصليبي وا...,العرب البقرة الحلوب للمستعمر الغربي الصليبي وا...,"[العرب, البقرة, الحلوب, للمستعمر, الغربي, الصل..."


# Splitting Data into Test and Train


In [108]:
#Now we split our data set into train and test. We will use 90 % data for training and 10 % for testing.
!pip install scikit-learn as sklearn
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data, test_size=0.10, random_state=42)

In [109]:
#Then we build training vocabulary and get maximum training sentence length and total number of words training data.

all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

109649 words total, with a vocabulary size of 36561
Max sentence length is 52


In [110]:
#Then we build testing vocabulary and get maximum testing sentence length and total number of words in testing data.
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

12253 words total, with a vocabulary size of 7219
Max sentence length is 47


In [112]:
#Using pre-trained word embeddings 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence

tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())
train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))
train_cnn_data = sequence.pad_sequences(training_sequences, maxlen=None)

Found 36561 unique tokens.


In [114]:
def rnn(embeddings, 
        max_sequence_length, 
        num_words, 
        embedding_dim, 
        labels_index):
    
    embedding_layer = Embedding(num_words,
                                embedding_dim,
                                weights=[embeddings],
                                input_length=max_sequence_length,
                                trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,),
                                  dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    lstm = LSTM(256)(embedded_sequences)
    
    x = Dense(128, activation='relu')(lstm)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [116]:
model = rnn(train_embedding_weights, 
            MAX_SEQUENCE_LENGTH, 
            len(train_word_index)+1, 
            EMBEDDING_DIM, 
            len(list(label_names)))

NameError: ignored